In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# LCEL (LangChain Expression Language)

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "basketball"})

'Why did the basketball go to therapy? Because it had too many dribbling issues!'

#### Breaking it down

In [3]:
prompt_value = prompt.invoke({"topic": "basketball"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about basketball')])

In [4]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about basketball')]

In [5]:
#Chat model outputs BaseMessage
message = model.invoke(prompt_value)
message

AIMessage(content='Why was the basketball court always so hot? Because all the players kept shooting!')

In [6]:
from langchain_openai.llms import OpenAI

#LLM model outputs string
llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nWhy did the basketball player go to the doctor?\n\nBecause he had court-itis!'

In [7]:
output_parser.invoke(message)

'Why was the basketball court always so hot? Because all the players kept shooting!'

# Retrieval augmented generation chain

In [18]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

#### Breaking it down

In [21]:
retriever.invoke("What do bears like to eat?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [22]:
(setup_and_retrieval | prompt).invoke("What do bears like to eat?")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(page_content='bears like to eat honey'), Document(page_content='harrison worked at kensho')]\n\nQuestion: What do bears like to eat?\n")])